In [1]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv
/kaggle/input/crime-cast-forecasting-crime-categories/train.csv
/kaggle/input/crime-cast-forecasting-crime-categories/test.csv


**Exploratory Data Analysis**

In [2]:
train = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/train.csv')
test = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/test.csv')
sample = pd.read_csv('/kaggle/input/crime-cast-forecasting-crime-categories/sample.csv')

# Preprocess dates
X = train.drop('Crime_Category', axis=1)
y = train['Crime_Category']

In [3]:
X.describe()

,Latitude,Longitude,Time_Occurred,Area_ID,Reporting_District_no,Part 1-2,Victim_Age,Premise_Code,Weapon_Used_Code
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,7335.000000
mean,33.940704,-117.893072,1352.380350,10.834250,1129.599200,1.418150,30.135000,297.176950,366.405044
std,2.126810,7.377726,646.100291,6.033166,603.330519,0.493267,21.863645,212.007298,120.485687
min,0.000000,-118.663400,1.000000,1.000000,101.000000,1.000000,-2.000000,101.000000,101.000000
25%,34.009200,-118.429700,930.000000,6.000000,632.000000,1.000000,12.000000,101.000000,312.000000
50%,34.058700,-118.323000,1430.000000,11.000000,1162.000000,1.000000,31.000000,203.000000,400.000000
75%,34.165025,-118.274400,1900.000000,16.000000,1622.000000,2.000000,46.000000,501.000000,400.000000
max,34.328100,0.000000,2359.000000,21.000000,2197.000000,2.000000,99.000000,969.000000,515.000000


In [4]:
X.shape

(20000, 21)

In [5]:
X.head()

,Location,Cross_Street,Latitude,Longitude,Date_Reported,Date_Occurred,Time_Occurred,Area_ID,Area_Name,Reporting_District_no,...,Modus_Operandi,Victim_Age,Victim_Sex,Victim_Descent,Premise_Code,Premise_Description,Weapon_Used_Code,Weapon_Description,Status,Status_Description
0,4500 CARPENTER AV,NaN,34.1522,-118.3910,03/09/2020 12:00:00 AM,03/06/2020 12:00:00 AM,1800.0,15.0,N Hollywood,1563.0,...,0385,75.0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont
1,45TH ST,ALAMEDA ST,34.0028,-118.2391,02/27/2020 12:00:00 AM,02/27/2020 12:00:00 AM,1345.0,13.0,Newton,1367.0,...,0906 0352 0371 0446 1822 0344 0416 0417,41.0,M,H,216.0,SWAP MEET,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont
2,600 E MARTIN LUTHER KING JR BL,NaN,34.0111,-118.2653,08/21/2020 12:00:00 AM,08/21/2020 12:00:00 AM,605.0,13.0,Newton,1343.0,...,0329 1202,67.0,M,B,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont
3,14900 ORO GRANDE ST,NaN,34.2953,-118.4590,11/08/2020 12:00:00 AM,11/06/2020 12:00:00 AM,1800.0,19.0,Mission,1924.0,...,0329 1300,61.0,M,H,101.0,STREET,NaN,NaN,IC,Invest Cont
4,7100 S VERMONT AV,NaN,33.9787,-118.2918,02/25/2020 12:00:00 AM,02/25/2020 12:00:00 AM,1130.0,12.0,77th Street,1245.0,...,0416 0945 1822 0400 0417 0344,0.0,X,X,401.0,MINI-MART,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont


In [6]:
X.dtypes

Location                  object
Cross_Street              object
Latitude                 float64
Longitude                float64
Date_Reported             object
Date_Occurred             object
Time_Occurred            float64
Area_ID                  float64
Area_Name                 object
Reporting_District_no    float64
Part 1-2                 float64
Modus_Operandi            object
Victim_Age               float64
Victim_Sex                object
Victim_Descent            object
Premise_Code             float64
Premise_Description       object
Weapon_Used_Code         float64
Weapon_Description        object
Status                    object
Status_Description        object
dtype: object

In [7]:


# Preprocess dates

X['Date_Reported'] = pd.to_datetime(X['Date_Reported'], format='%m/%d/%Y %I:%M:%S %p')
X['Date_Occurred'] = pd.to_datetime(X['Date_Occurred'], format='%m/%d/%Y %I:%M:%S %p')
test['Date_Reported'] = pd.to_datetime(test['Date_Reported'], format='%m/%d/%Y %I:%M:%S %p')
test['Date_Occurred'] = pd.to_datetime(test['Date_Occurred'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date components for both train and test data
for df in [X, test]:
    df['Reported_Year'] = df['Date_Reported'].dt.year
    df['Reported_Month'] = df['Date_Reported'].dt.month
    df['Reported_Day'] = df['Date_Reported'].dt.day
    df['Occurred_Year'] = df['Date_Occurred'].dt.year
    df['Occurred_Month'] = df['Date_Occurred'].dt.month
    df['Occurred_Day'] = df['Date_Occurred'].dt.day


X = X.drop(['Date_Reported', 'Date_Occurred'], axis=1)
test = test.drop(['Date_Reported', 'Date_Occurred'], axis=1)
# Handle negative ages
median_age = X[X['Victim_Age'] >= 0]['Victim_Age'].median()
X['Victim_Age'] = X['Victim_Age'].apply(lambda x: median_age if x < 0 else x)
median_age_test = test[test['Victim_Age'] >= 0]['Victim_Age'].median()
test['Victim_Age'] = test['Victim_Age'].apply(lambda x: median_age_test if x < 0 else x)

# Identify columns
categorical_columns = X.select_dtypes(include=['object']).columns
numeric_columns = X.select_dtypes(include=[np.number]).columns

# Define transformers
numeric_transformer = Pipeline(steps=[
    ('meanimputer', SimpleImputer(strategy='mean')),
    ('standardscaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('frequentimputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer, numeric_columns),
    ('categoric', categorical_transformer, categorical_columns)
])

In [8]:
# Logistic Regression Model
def logistic_regression_model(X_train, y_train):
    logistic_regression = LogisticRegression(max_iter=2000, solver='liblinear')
    feature_selector = SelectFromModel(logistic_regression, threshold='mean')
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('feature_selection', feature_selector),
        ('classifier', logistic_regression)
        
    ])
    param_grid = {
        'classifier__C': [0.1, 1, 10],
        'classifier__penalty': ['l1', 'l2']
    }
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    return grid_search


In [9]:
# KNN Model
def knn_model(X_train, y_train):
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', KNeighborsClassifier())
    ])
    param_grid = {
        'classifier__n_neighbors': [3, 5, 7, 9],
        'classifier__weights': ['uniform', 'distance'],
        'classifier__metric': ['euclidean', 'manhattan', 'minkowski']
    }
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    return grid_search

In [10]:
# Random Forest Model
def random_forest_model(X_train, y_train):
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=7))
    ])
    
    param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5],
    'classifier__min_samples_leaf': [1, 2],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__bootstrap': [True, False],
    'classifier__class_weight': [None, 'balanced']
}

    
    # Use StratifiedKFold to ensure balanced class distribution in each fold
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
    
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)
    
    print("Best parameters found: ", grid_search.best_params_)
    print("Best cross-validation score: ", grid_search.best_score_)
    
    return grid_search



In [11]:
def make_predictions(model_func, X_train, X_valid, y_train, y_valid, test):
    grid_search = model_func(X_train, y_train)
    y_valid_pred = grid_search.predict(X_valid)
    print("Validation Accuracy: ", accuracy_score(y_valid, y_valid_pred))
    print("Best parameters found: ", grid_search.best_params_)
    print("Best cross-validation score: ", grid_search.best_score_)
    
    # Predict on the test set
    best_model = grid_search.best_estimator_
    Y_test_pred = best_model.predict(test)
    
    # Prepare the submission
    submission = pd.DataFrame({
        "ID": np.arange(1, len(Y_test_pred) + 1),
        "Crime_Category": Y_test_pred
    })
    submission.to_csv('submission.csv', index=False)
    return submission

# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=7)

# Choose which model to run: logistic_regression_model, knn_model, random_forest_model
submission = make_predictions(logistic_regression_model, X_train, X_valid, y_train, y_valid, test)


Validation Accuracy:  0.8711666666666666
Best parameters found:  {'classifier__C': 1, 'classifier__penalty': 'l1'}
Best cross-validation score:  0.8768571428571429
